# Gradient decent

## Exact Gradient Computation

Given a function f, we can compute its exact gradient at any x if f's derivative is easy to compute. For example, let $f(x)=2x^2-3x+\ln x$, where $x>0$.

## Numerical Gradient Computation

Instead of computing the derivative of a function, we can also estimate the gradient numerically with various methods. These methods are essential, especially when a callable function is not exposed due to privacy reasons, or it is hard to differentiate analytically. 

To numerically compute the gradient, the simple way is to follow Newton's difference quotient: $f'(x)=\lim_{h\to 0}{f(x+h)-f(x)\over h}$. Another two-point formula is to compute the slope through the points $(x-h,f(x-h))$ and $(x+h,f(x+h))$. Let us reuse the example function $f(x)=2x^2-3x+\ln x$ and test the precision of these two approaches. We define the function in the next cell, and try to compute its gradient via both methods at $x=2$. Range h value in [0.1,0.01,0.001,0.0001] and report all gradients calculated.

In [1]:
import math

def f(x):
    return (2*(x**2) - (3*x) + math.log(x))

In [3]:
# Compute gradient using the first method (Newton's difference quotient)
def grad_1(x, h):
    return (f(x+h) - f(x)) / h
#print gradients from h in [0.1, 0.01, 0.001, 0.0001, 0.00001]
print ("Gradient using the first method (Newton's difference quotient)")
for h in [0.1, 0.01, 0.001, 0.0001, 0.00001]:
    print("h="+str(h)+"; f'(2)="+str(grad_1(2, h)))

Gradient using the first method (Newton's difference quotient)
h=0.1; f'(2)=5.687901641694317
h=0.01; f'(2)=5.518754151103744
h=0.001; f'(2)=5.50187504165045
h=0.0001; f'(2)=5.5001875004201395
h=1e-05; f'(2)=5.500018749993173


In [5]:
# Compute gradient using the second method 
def grad_2(x, h):
    return (f(x+h) - f(x-h)) / (2*h)
#print gradients from h in [0.1, 0.01, 0.001, 0.0001, 0.00001]
print ("Gradient using the second method (Two-point difference quotient)")
for h in [0.1, 0.01, 0.001, 0.0001, 0.00001]:
    print("h="+str(h)+"; f'(2)="+str(grad_2(2, h)))

Gradient using the second method (Two-point difference quotient)
h=0.1; f'(2)=5.500417292784909
h=0.01; f'(2)=5.500004166729067
h=0.001; f'(2)=5.500000041665842
h=0.0001; f'(2)=5.500000000417948
h=1e-05; f'(2)=5.499999999991622


It seems like the second method is more accurate for the given equation because it was very close to the actual derivative at $x=2$ even with the first value of h. However, more trials with more equations and different values of h would be needed to definitively say which one is more accurate in general. 

## Logistic regression

Logistic regression is a classification tool that models the probability of an event taking place by having the log odds for the event be a linear combination of one or more independent variables. Specifically, let $\vec{x}=<x_1,\dots ,x_m>$ be an m dimensional vector of independent variables (features), and $y$ be the corresponding binary dependent variable (label). The probability of having $y=1$ is modeled as $$P_y={1\over 1+e^{-(b_0+b_1\cdot x_1+\dots +b_m\cdot x_m)}}={1\over 1+e^{-(b_0+\vec{b}_{1:m}\cdot\vec{x})}}$$

Given a set of data points $<\vec{x}_k,y_k>$ with $k\in [1,n]$, how can we fit the model with these data, i.e., how to choose the best $\vec{b}=b_0,b_1\cdots,b_m$?

One way is to write out the likelihood $$\prod_{k:y_k=1}P_{y_k}\prod_{k:y_k=0}(1-P_{y_k})$$ and find $b_0,b_1\cdots,b_m$ that maximize its logarithm, $$l=\sum_{k:y_k=1}\ln(P_{y_k})+\sum_{k:y_k=0}\ln(1-P_{y_k})$$

In contrast to computing the closed form gradient of a Least-squares loss in a linear model, doing the same for logistic regression is not possible. Gradient descent can be used to optimize such function $l$, and we will implement it step-by-step. First, we write a function log_likelihood in the next cell that computes the log-likelihood given data points and $\vec{b}$. [5 pts]

In [10]:
import numpy as np
import sklearn

In [16]:
def log_likelihood(X,y,b):
    """
    X: n*m numpy data array.
    y: one dimension numpy data array of length n
    b: one dimension numpy data array of length m+1
    
    Return the log likelihood.
    """
    #commpute P_yk
    p = ([[0, 0] for i in range(len(y)) ])
    for i in range(len(y)):
        p[i][0] = 1/(1+np.exp(-1*(b[0]+np.dot(X[i],b[1:]))))
        p[i][1] = 1-p[i][0]
    #compute log likelihood
    log_likelihood = 0
    for i in range(len(y)):
        if y[i] == 1:
            log_likelihood += np.log(p[i][0])
        else:
            log_likelihood += np.log(p[i][1])
    return log_likelihood
   

In [57]:
def compute_gradient(X,y,b):
# The inputs are the same as the ones of log_likelihood
# Return the gradient of the log likelihood with respect to vector b
    def f(x):
        return log_likelihood(X,y,x)
    def grad_1(x, h):
        grad=[0 for _ in range(len(x))]
        for i in range(len(x)):
            h2=[0 for _ in range(len(x))]
            h2[i]=h 
            grad[i]=(f(x+h2)-f(x))/h
        return np.array(grad)
    return grad_1(b, 0.00001)
    

In [58]:
import scipy.optimize as opt
# Test your function here, preserve the output
compute_gradient(X,y,b)



array([-0.8785005 , -0.09478745])

Once we know how to compute the gradients, we can optimize the objective, which is log-likelihood in our case, using gradient descent. It iteratively changes the parameters in a small "step" towards the gradient direction, i.e., the direction where the objective increases at the fastest pace. Formally, denote the calculated gradients as $\Delta (\vec{b})$, we can update our parameters via $\vec{b}=\vec{b}+\gamma \cdot \Delta (\vec{b})$, where $\gamma$ is the size of the "step". We repeat this process until the objective stop improving or a pre-set max number of iterations is reached. 

In [83]:
def gradient_descent(X, y, initial_b, step_size, max_iteration,prin=False):
    """
    X: n*m numpy data array.
    y: one dimension numpy data array of length n
    initial_b: one dimension numpy data array of length m+1
    step_size: scalar, the size of one step update
    max_iteration: scalar, the max number of iterations
    Return the updated coefficient vector b.
    """
    b = initial_b
    c=max_iteration
    for i in range(max_iteration):
        grad=compute_gradient(X,y,b)
        length=np.linalg.norm(grad)
        if length > 1e-8:
            b=b+(step_size*grad/length)
        else: 
            c=i
            break
    if(prin):
        print("final number of iterations: "+str(c))
    return b

In [86]:
optimized_b = gradient_descent(X, y, b, 0.1, 1000)
print(optimized_b)

[-52.42477565  70.06060629]


Next, we apply the implemented logistic regression model to a real dataset. The dataset is a trimmed breast-cancer-Wisconsin dataset from UCI machine learning Repository. Only 100 data points are offered in the training set to make sure the computation can be finished swiftly. The training dataset is loaded in the next cell, and the vector $\vec{b}$ is also randomly initialized. 

Fit three models with the training set using different step size ranging in [0.01,0.05,0.1] and set the max number of iterations as 10000.

In [87]:
f = open("breast-cancer-wisconsin.data","r")
X_train = []
y_train = []
for line in f:
    tmp = []
    for part in line.strip().split(",")[1:-1]:
        tmp.append(float(part))
    y_train.append((0 if line.strip().split(",")[-1]=="2" else 1))
    X_train.append(tmp)
X_train = np.array(X_train)
y_train = np.array(y_train)
random_b = np.random.uniform(0,1,size=(10))

In [89]:
# Fit three models with different step size, report the final log-likelihood, 
# number of iterations and the final coefficent vector b.
b=[0,0,0]
i=0
for h in (.01, .05, .1):
    print("step size: "+str(h))
    b[i]=gradient_descent(X_train, y_train, random_b, h, 10000, prin=True)
    print("final log-likelihood: "+str(log_likelihood(X_train, y_train, b[i]))+"\n")
    i+=1

step size: 0.01
final number of iterations: 10000
final log-likelihood: -7.187028615249007

step size: 0.05
final number of iterations: 10000
final log-likelihood: -7.247861048924458

step size: 0.1
final number of iterations: 10000
final log-likelihood: -7.626632940942781



The final log-likelihoods got smaller as the step size increased in length. However, the number of iterations remained the same (the max value).

In [90]:
f = open("test_data.txt","r")
X_test = []
y_test = []
for line in f:
    tmp = []
    for part in line.strip().split(",")[1:-1]:
        tmp.append(float(part))
    y_test.append((0 if line.strip().split(",")[-1]=="2" else 1))
    X_test.append(tmp)

In [97]:
# Predict based on your models and report the accuracy
results=[[0,0,0] for _ in range(len(y_test))]
prediction=[[0,0,0] for _ in range(len(y_test))]
for i in range(len(results)):
    results[i][0]= 1/(1+np.exp(-1*(b[0][0]+np.dot(X_test[i],b[0][1:]))))
    results[i][1]= 1/(1+np.exp(-1*(b[1][0]+np.dot(X_test[i],b[1][1:]))))
    results[i][2]= 1/(1+np.exp(-1*(b[2][0]+np.dot(X_test[i],b[2][1:]))))
    prediction[i][0]=1 if (results[i][0]) >.5 else 0
    prediction[i][1]=1 if (results[i][1]) >.5 else 0
    prediction[i][2]=1 if (results[i][2]) >.5 else 0
accuracy=[0,0,0]
for i in range(len(y_test)):
    if prediction[i][0]==y_test[i]:
        accuracy[0]+=1
    if prediction[i][1]==y_test[i]:
        accuracy[1]+=1
    if prediction[i][2]==y_test[i]:
        accuracy[2]+=1
accuracy[0]/=len(y_test)
accuracy[1]/=len(y_test)
accuracy[2]/=len(y_test)
print("accuracy for step size .01: "+str(accuracy[0]))
print("accuracy for step size .05: "+str(accuracy[1]))
print("accuracy for step size .1: "+str(accuracy[2]))


    


accuracy for step size .01: 0.9
accuracy for step size .05: 0.9
accuracy for step size .1: 0.9
